In [1]:
root_dir = '..' # dir of the source code
import sys
sys.path.append(root_dir)
from train import main
import numpy as np

In [2]:
class config:
    data = 'ACM'
    data_path = '../data/preprocessed/'
    model = 'HAN'
    target_node_type = 'paper'
    n_hid = 64
    num_heads = 8
    dropout = 0.4
    learning_rate = 0.005
    weight_decay = 1e-4
    patience = 100
    num_iter = 500
    num_test = 30
    hop = 20
    filter_pct = 0.1 # remove the top and bottom filer_pct points before obtaining statistics of test accuracy
    log_step = 1000 # training log step
    average = 'macro' # 'f1 average: can choose either macro or micro.'

args = config()

In [3]:
f1s = []
datasets = ['ACM', 'IMDB', 'DBLP']
for dataset in datasets:
    args.data = dataset
    if dataset == 'ACM':
        args.target_node_type = 'paper'
        print('\nstart testing on ' + dataset + ' dataset')
        f1s.append(main(args))
    elif dataset == 'IMDB':
        args.target_node_type = 'movie'
        print('\nstart testing on ' + dataset + ' dataset')
        f1s.append(main(args))
    elif dataset == 'DBLP':
        args.num_heads = 1
        args.target_node_type = 'author'
        print('\nstart testing on ' + dataset + ' dataset')
        f1s.append(main(args))


start testing on ACM dataset
total time = 229.621, train time/epoch = 0.78600, best_val_f1 (macro) = 0.287, test_f1 (macro) = 0.356
total time = 121.208, train time/epoch = 0.78635, best_val_f1 (macro) = 0.485, test_f1 (macro) = 0.453
total time = 115.272, train time/epoch = 0.78470, best_val_f1 (macro) = 0.167, test_f1 (macro) = 0.234
total time = 115.449, train time/epoch = 0.78530, best_val_f1 (macro) = 0.167, test_f1 (macro) = 0.117
total time = 115.127, train time/epoch = 0.78278, best_val_f1 (macro) = 0.515, test_f1 (macro) = 0.564
total time = 115.425, train time/epoch = 0.78512, best_val_f1 (macro) = 0.167, test_f1 (macro) = 0.131
total time = 120.109, train time/epoch = 0.78653, best_val_f1 (macro) = 0.222, test_f1 (macro) = 0.176
total time = 138.011, train time/epoch = 0.78546, best_val_f1 (macro) = 0.522, test_f1 (macro) = 0.578
total time = 132.064, train time/epoch = 0.78365, best_val_f1 (macro) = 0.212, test_f1 (macro) = 0.268
total time = 115.314, train time/epoch = 0.

total time = 154.677, train time/epoch = 0.74596, best_val_f1 (macro) = 0.100, test_f1 (macro) = 0.099
total time = 154.706, train time/epoch = 0.74428, best_val_f1 (macro) = 0.100, test_f1 (macro) = 0.067
total time = 154.795, train time/epoch = 0.74450, best_val_f1 (macro) = 0.100, test_f1 (macro) = 0.119
total time = 154.781, train time/epoch = 0.74518, best_val_f1 (macro) = 0.100, test_f1 (macro) = 0.067
total time = 154.739, train time/epoch = 0.74430, best_val_f1 (macro) = 0.100, test_f1 (macro) = 0.099
total time = 154.661, train time/epoch = 0.74366, best_val_f1 (macro) = 0.100, test_f1 (macro) = 0.067
total time = 154.693, train time/epoch = 0.74427, best_val_f1 (macro) = 0.100, test_f1 (macro) = 0.099
total time = 154.746, train time/epoch = 0.74461, best_val_f1 (macro) = 0.100, test_f1 (macro) = 0.119
total time = 154.724, train time/epoch = 0.74506, best_val_f1 (macro) = 0.100, test_f1 (macro) = 0.110
total time = 154.633, train time/epoch = 0.74392, best_val_f1 (macro) = 0

In [4]:
f1s = np.array(f1s)
np.savetxt('f1_ACM_hop20.txt',f1s)

In [9]:
from utils import remove_edge_pts
# remove some unstable results
f1_ACM = f1s[0][f1s[0] > 0.2]
f1 = remove_edge_pts(f1_ACM, pct=args.filter_pct)
f1.mean(), f1.std()

(0.439253551597688, 0.10978656089867266)

In [5]:
# remove anomaly points with value <= 0.7
f1_DBLP = f1s[-1]
f1_DBLP = f1_DBLP[f1_DBLP > 0.7]

In [6]:
def remove_edge_pts(accs, pct=0.1):
    accs = sorted(list(accs))
    N = len(accs)
    M = int(N * pct)
    accs = np.array(accs[M:N-M])
    return accs